In [1]:
import os
os.environ['LDFLAGS'] = "-L/opt/homebrew/opt/libomp/lib"
os.environ['CPPFLAGS'] = "-I/opt/homebrew/opt/libomp/include"
# !pip3 install cmake
# !pip3 install --no-cache-dir lightgbm
# !pip3 install lightgbm

In [12]:
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import os
from pathlib import Path
# !pip3 install pandas numpy seaborn matplotlib lightgbm tqdm pickle5
import pandas as pd

import pickle as pk
from tqdm import tqdm
import gc
from pathlib import Path

warnings.filterwarnings("ignore")
sys.path.append("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/") # path to the `src`` folder
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from src.data import DataHelper
# from src.data import DataHelper_Vibrent

from src.retrieval.rules import SaleTrendTopN


In [6]:
# data_dir = Path("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/data/Vibrent")
# model_dir = Path("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/models/models_Vibrent")
data_dir = Path("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/data/HM")
model_dir = Path("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/models/models_HM")

In [7]:
TRAIN_WEEK_NUM = 4
WEEK_NUM = TRAIN_WEEK_NUM + 2

VERSION_NAME = "Recall NUser"
TEST = True # * Set as `False` when do local experiments to save time

In [8]:
import os
if not os.path.exists(data_dir/"interim"/VERSION_NAME):
    os.mkdir(data_dir/"interim"/VERSION_NAME)
if not os.path.exists(data_dir/"processed"/VERSION_NAME):
    os.mkdir(data_dir/"processed"/VERSION_NAME)

Pepare data: encoding ids and preprocessing

In [9]:
dh = DataHelper(data_dir)
# dh = DataHelper_Vibrent(data_dir)

## Retrieval


In [ ]:
# data = dh.preprocess_data(save=True, name="encoded_full") # * run only once, processed data will be saved

In [10]:
data = dh.load_data(name="encoded_full")

In [13]:
listBin = [-1, 19, 29, 39, 49, 59, 69, 119]
data['user']['age_bins'] = pd.cut(data['user']['age'], listBin)

In [16]:
trans = data["inter"]

retriever = SaleTrendTopN(trans_df=trans, days=7, n=12)
top_trending_df = retriever.retrieve()

with open(data_dir/"index_id_map"/"item_index2id.pkl", "rb") as f:
    index2id = pk.load(f)

top_trending_df["article_id"] = top_trending_df["article_id"].map(index2id)

top_trending_df["article_id"] = top_trending_df["article_id"].astype(str).str.zfill(10)

top_trending_df.to_csv("top_12_trending_products_mapped.csv", index=False)
